<a href="https://colab.research.google.com/github/napoles-uach/streamlit_apps/blob/main/Streamlit_Colab/03_Streamlit__Colab_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial 3. Official Demo of Streamlit. 
## This notebook shows how to deploy an Streamlit app on top of Google Colaboratory.

![](https://github.com/napoles-uach/streamlit_apps/blob/main/Streamlit_Colab/Images/03.png?raw=True)

## 1) Run all and click the "**Link to web app**" (ignore the output of the last cell).
## 2) Modify the code in `app.py`


# Setup

Installation of Streamlit of course!!

In [ ]:
!pip install streamlit -q

Here we use the ngrok cloud service. Don't change this!!

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -qq ngrok-stable-linux-amd64.zip

# Sometimes you have to run the cell below a couple of times to get the link to the app.
Use the output here!!

In [ ]:
print('Link to web app')
get_ipython().system_raw('./ngrok http 8501 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

The following is the `app.py` base code. It can be modified in the cell or in the folder to the left.

In [5]:
%%bash
cat > app.py <<EOF
import streamlit as st
import pandas as pd
import numpy as np

st.title('Uber pickups in NYC')

DATE_COLUMN = 'date/time'
DATA_URL = ('https://s3-us-west-2.amazonaws.com/'
            'streamlit-demo-data/uber-raw-data-sep14.csv.gz')

@st.cache
def load_data(nrows):
    data = pd.read_csv(DATA_URL, nrows=nrows)
    lowercase = lambda x: str(x).lower()
    data.rename(lowercase, axis='columns', inplace=True)
    data[DATE_COLUMN] = pd.to_datetime(data[DATE_COLUMN])
    return data

data_load_state = st.text('Loading data...')
data = load_data(10000)
data_load_state.text("Done! (using st.cache)")

if st.checkbox('Show raw data'):
    st.subheader('Raw data')
    st.write(data)

st.subheader('Number of pickups by hour')
hist_values = np.histogram(data[DATE_COLUMN].dt.hour, bins=24, range=(0,24))[0]
st.bar_chart(hist_values)

# Some number in the range 0-23
hour_to_filter = st.slider('hour', 0, 23, 17)
filtered_data = data[data[DATE_COLUMN].dt.hour == hour_to_filter]

st.subheader('Map of all pickups at %s:00' % hour_to_filter)
st.map(filtered_data)
# Put your Python+Streamlit code here ...
# you can modify it by double cliking on the folder icon at the left


EOF

This last cell would keep the app running. If stoped, the app woul be disconnected.

In [ ]:
!streamlit run app.py